# Interpretação e Modelagem direta MT 1-D

### Este programa calcula o Tensor de impedância em função da frequencia para um modelo de terra 1-D e compara com um conjunto de dados observados fornecidos pelo usuário.

In [ ]:
#Importar bibliotecas

import numpy as np
from matplotlib import pyplot as plt
from matplotlib import gridspec as gridspec

In [ ]:
%matplotlib inline

### Definindo a função dos parâmetros auxiliares e do Tensor de Impedância

In [ ]:
def Auxiliars(freq, p0,h):
    
    ''' This function calculates the auxiliars W, gamma, R and
    Impedance Tensor (Z). The entry parameters are frequency (freq) 
    and parameter vector (p0) in resistivity.
    This function returns vectors gamma, w, e and Z which contains 
    the respectivity values for each layer of p0
    
    Inputs:
    freq = frequency (Hz)
    p0 = vector of resistivities'''
    
    M = len(h) # Number of layers
    
    Z = np.zeros(M, dtype = 'complex') 
    gamma = np.zeros(M, dtype = 'complex')
    w = np.zeros(M, dtype = 'complex')
    R = np.zeros(M, dtype = 'complex')
    e = np.zeros(M, dtype = 'complex')

    gamma = np.sqrt((freq*mu0*im)/p0)
    w = gamma*p0
    e = np.exp(-2.0*gamma*h)   # Elemento e[-1] não é utilizado
    
    # Impedance at last layer
    
    gamma[-1] = np.sqrt((freq*mu0*im)/p0[-1])
    w[-1] = gamma[-1]*p0[-1]
    Z[-1] = w[-1]
    e[-1] = 0.0 + 0.0j
        
    # Impedance at each layer
    for i in range(M-1):
        j = M-i-2
        R[j] = (w[j] - Z[j+1])/(w[j] + Z[j+1])
        Z[j] = w[j] * ((1.0-R[j]*e[j])/(1.0+R[j]*e[j]))
 
    return gamma, w, R, e, Z

def Impedance_Tensor(omega, p0,h):
    '''This function calculates the impedance tensor for a given vector of frequencies (omega) and a given vector of resitivities (p0)
    
    Inputs:
    
    omega = vector of frequencies
    p0 = vector of resistivities'''
    N = len(omega)
    Zcalc = np.zeros(N, dtype = 'complex')
    
    for k in range(N):
        freq = omega[k]
        gamma, w, R, e, Z = Auxiliars(freq, p0,h)
        Zcalc[k] = Z[0]
    
    return Zcalc

## Abre o arquivo de dados

In [ ]:
period, rhoap_obs, phi_obs, Zobsreal, Zobsimag = np.loadtxt("data.txt",skiprows=1, delimiter=" ",unpack=True)

#freq, rhoap_obs, phi_obs = np.loadtxt("SF01Xa.txt",skiprows=3, delimiter=" ",unpack=True,usecols=(0,1,3))
#period = 1/freq

### Plota os dados

In [ ]:
plt.figure(figsize=(5,8))

plt.subplot(2,1,1)
plt.plot(period,rhoap_obs,'ro')
plt.xscale('log')
plt.yscale('log')
plt.ylim([10,10000.])
plt.xlim([np.min(period),np.max(period)])
plt.ylabel('Apparent Resistivity (ohm.m)',fontsize='x-large')

plt.subplot(2,1,2)
plt.plot(period,phi_obs,'ro')
plt.xscale('log')
plt.xlabel('Period (sec)',fontsize='x-large')
plt.ylabel('Phase (degree)',fontsize='x-large')
plt.ylim([0,90])

plt.show()

### Modelo de terra 1-D

#### Defina o vetor h (espessuras das camadas) e o vetor p0 (resistividades das camadas)

In [ ]:
h = np.array([100., 100., 100., 100., 100., 100.], dtype = 'float') # Vetor de espessuras (m)
p0 = np.array([100.0, 100., 100., 100.0, 100.0, 100.], dtype = 'float') # Vetor de resistividades (Ohm.m)

In [ ]:
## DEFINIÇAO DE PARAMETROS

# permeabilidade magnetica
mu0 = 4*np.pi*1E-7 

# Frequência (Hz)
f = 1./period
omega = 2.*f*np.pi # frequencia angular

N = len(omega) # Número de dados
M = len(p0) # Número de parâmetros
z0 = 0  # Profundidade do topo da primeira camada
im = (0.0+1.0j) #número imaginário

### Cálculo do Tensor de Impedância

In [ ]:
Zcalc = Impedance_Tensor(omega,p0,h)

### Cálculo da resistividade aparente e fase

In [ ]:
rhoap = (((omega*mu0)**-1.0)*(abs(Zcalc)**2))
phase = np.arctan(Zcalc.imag/Zcalc.real)
phi = (phase*180)/np.pi

### Plot das curvas de resitividade aparente e fase

In [ ]:
#Calculation of depths for each interface

interfaces = np.zeros(M)
for l in range(M):
    interfaces[l] = z0 + sum(h[:l])

In [ ]:
fig = plt.figure(figsize=(14,10))
gs2 = gridspec.GridSpec(3, 3, width_ratios=[2, 4, 4])
gs2.update(left=5, right=6, hspace=0.5)

ax1 = plt.subplot(gs2[:, :-2])
ax1.step(p0, interfaces, 'r')
plt.xscale('log')
ax1.set_xlabel('Resistivity (Ohm.m)',fontsize='x-large')
ax1.set_ylabel('Depth(m)',fontsize='x-large')
ax1.set_xlim((10,10000))
ax1.set_ylim((np.max(interfaces)+100, 0))
ax1.set_title('1-D Resistivity Model',fontsize='x-large')
ax1.tick_params(labelsize=14)

ax2 = plt.subplot(gs2[:-1, -2])
ax2.plot(period, rhoap_obs,'ro', label='observed')
ax2.plot(period, rhoap,'r-', label='calculated')
plt.legend(fontsize='x-large',numpoints = 1)
plt.xscale('log')
plt.yscale('log')
plt.ylim([10,10000.])
plt.xlim([np.min(period),np.max(period)])
plt.xlabel('Period (sec)',fontsize='x-large')
plt.ylabel('Apparent Resistivity (ohm.m)',fontsize='x-large')
ax2.tick_params(labelsize=14)

ax3 = plt.subplot(gs2[-1, -2])
ax3.plot(period,phi_obs, 'ro', label='observed')
ax3.plot(period,phi, 'r-', label='calculated')
plt.legend(fontsize='x-large',numpoints = 1)
plt.xscale('log')
plt.xlabel('Period (sec)',fontsize='x-large')
plt.ylabel('Phase (degree)',fontsize='x-large')
plt.ylim([0,90])
ax3.tick_params(labelsize=14)